In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation

from sklearn.preprocessing import LabelEncoder

#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf
import seaborn as sns

from pathlib import Path
from warnings import simplefilter
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from statsmodels.tsa.seasonal import seasonal_decompose
from keras_self_attention import SeqSelfAttention

In [ ]:
!pip install keras-self-attention

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
data = pd.read_csv('/content/drive/MyDrive/Data/dataTS.csv', 
                   usecols = ['Time', 'pH', 'DO', 'TSS', 'TN', 'TP', 'TOC', 'ORP', 'Temp',
       'TEMP'], index_col= 'Time', parse_dates=['Time']).reindex(columns=['pH', 'DO', 'TSS', 'TN', 'TP', 'TOC', 'ORP', 'Temp',
       'TEMP'])
# data["Date_time"] = data.index.strftime('%H:%M:%S')
# data["Month"] = data.index.month
# data["Day"] = data.index.day
# for colname in data.select_dtypes(["object", "category"]):
#     data[colname], _ = data[colname].factorize()

In [10]:
data_train, data_test = train_test_split(
    data, test_size= 100, shuffle=False)
data_train, data_val = train_test_split(
    data_train, test_size= 100, shuffle=False)

In [11]:
def generate_input_output_pairs(data, time_steps, target_steps):
    X = []
    y = []
    for i in range(len(data) - time_steps - target_steps + 1):
        X.append(data[i:i+time_steps,:])
        y.append(data[i+time_steps:i+time_steps+target_steps,:])
    return np.array(X), np.array(y)

In [12]:
X_train, y_train = generate_input_output_pairs(data_train.values, time_steps=20, target_steps=10)
X_val, y_val = generate_input_output_pairs(data_val.values, time_steps=20, target_steps=10)
X_test, y_test = generate_input_output_pairs(data_test.values, time_steps=20, target_steps=10)
# Add noise to data
noise_factor = 0.05
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)

In [16]:
# Define the number of features and time steps
num_features = 9
time_steps = 20
target_steps = 10

# Define the model
model = tf.keras.models.Sequential([
    # Encoder LSTM
    # tf.keras.layers.Normalization(input_shape=(time_steps, num_features)),
    tf.keras.layers.LSTM(32, input_shape=(time_steps, num_features),activation='relu', return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.LSTM(32, return_sequences=True),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.LSTM(1, activation='relu'),
    tf.keras.layers.RepeatVector(target_steps),
    SeqSelfAttention(attention_activation='softmax'),
    
    # Decoder LSTM
    tf.keras.layers.LSTM(32, activation='relu', return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64, return_sequences=True),
    
    # Output layer
    tf.keras.layers.TimeDistributed(Dense(num_features))
])

# Compile the model
model.compile(optimizer='adam', loss='mse')
model.summary()

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 repeat_vector (RepeatVector  (None, 10, 32)           0         
 )                                                               
                                                                 
 seq_self_attention (SeqSelf  (None, 10, 32)           2113      
 Attention)                                                      
                                                                 
 lstm_1 (LSTM)               (None, 10, 32)            8320      
                                                                 
 dropout_1 (Dropout)         (None, 10, 32)            0

In [17]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history = model.fit([X_train_noisy], [y_train],
                    validation_data=([X_val], [y_val]), 
                          batch_size=256, 
                          epochs=50, 
                          verbose=1, callbacks=[callback])

Epoch 1/50
1325/1325 [==============================] - 132s 94ms/step - loss: 22899.8789 - val_loss: 11740.9707
Epoch 2/50
1325/1325 [==============================] - 116s 87ms/step - loss: 17197.4219 - val_loss: 7250.2441
Epoch 3/50
1325/1325 [==============================] - 126s 95ms/step - loss: 13511.6406 - val_loss: 4421.4463
Epoch 4/50
1325/1325 [==============================] - 115s 87ms/step - loss: 10896.2236 - val_loss: 2678.8621
Epoch 5/50
1325/1325 [==============================] - 121s 91ms/step - loss: 9005.9951 - val_loss: 2228.6545
Epoch 6/50
1325/1325 [==============================] - 116s 88ms/step - loss: 7648.5654 - val_loss: 2156.1167
Epoch 7/50
1325/1325 [==============================] - 119s 90ms/step - loss: 6525.1924 - val_loss: 2044.3007
Epoch 8/50
1325/1325 [==============================] - 119s 90ms/step - loss: 5612.1748 - val_loss: 2026.8654
Epoch 9/50
1325/1325 [==============================] - 116s 88ms/step - loss: 5013.5020 - val_loss: 1941.0

In [18]:
y_pred = model.predict(X_test)
np.save('lstm_autoencoder_attention.npy', y_pred)

3/3 [==============================] - 1s 10ms/step


In [24]:
for i,j in zip(range(9),['pH', 'DO', 'TSS', 'TN', 'TP', 'TOC', 'ORP', 'Temp', 'TEMP']):
  print(j,' RMSE ',mean_squared_error(y_pred.reshape(-1,9)[:,i], y_test.reshape(-1,9)[:,i], squared = False))
  print(j,' MAE ',mean_absolute_error(y_pred.reshape(-1,9)[:,i], y_test.reshape(-1,9)[:,i]))

pH  RMSE  5.089031842117102
pH  MAE  5.078244631109103
DO  RMSE  1.945504490395827
DO  MAE  1.9332233102019403
TSS  RMSE  112.92961283195109
TSS  MAE  91.0061176380585
TN  RMSE  1.7323552653545509
TN  MAE  1.4168979648004003
TP  RMSE  58.913423054224594
TP  MAE  44.23245770793829
TOC  RMSE  9.366306772331729
TOC  MAE  7.499297641294686
ORP  RMSE  83.11849136242044
ORP  MAE  79.23437255687445
Temp  RMSE  2.3026838256149498
Temp  MAE  2.185959792016258
TEMP  RMSE  4.034697554160262
TEMP  MAE  3.383624367576869
